In [1]:
import os

os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [2]:
import sys

SOURCE_DIR = os.path.dirname(os.path.dirname(os.path.abspath(__name__)))
sys.path.insert(0, SOURCE_DIR)

In [3]:
import malaya_speech
import malaya_speech.config
from malaya_speech.train.model import conformer, wav2vec2
from malaya_speech.train.model.wav2vec2 import negative_sampling
import tensorflow as tf






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [4]:
config = malaya_speech.config.transducer_featurizer_config
featurizer = malaya_speech.utils.tf_featurization.STTFeaturizer(**config)

In [5]:
config = malaya_speech.config.conformer_base_encoder_config
base_model = conformer.Model(**config)

In [6]:
i = tf.placeholder(tf.float32, [None])
v = featurizer.vectorize(i)
length_v = tf.cast(tf.shape(v)[0], tf.int32)
length_v = tf.expand_dims(length_v, 0)
v = tf.expand_dims(v, 0)
v = tf.expand_dims(v, -1)
v, length_v

(<tf.Tensor 'ExpandDims_3:0' shape=(1, ?, 80, 1) dtype=float32>,
 <tf.Tensor 'ExpandDims_1:0' shape=(1,) dtype=int32>)

In [7]:
base_logits = base_model(v)
base_logits

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


<tf.Tensor 'conformer/conformer_block_15/conformer_block_15_ln/batchnorm/add_1:0' shape=(1, ?, 256) dtype=float32>

In [8]:
cfg = wav2vec2.Wav2Vec2Config(conv_feature_layers = '[(256,)]')
model = negative_sampling.Model(cfg)

In [9]:
r, num_vars, curr_temp = model(base_logits, base_logits)
r


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
dim is deprecated, use axis instead


{'x': <tf.Tensor 'wav2vec2/truediv:0' shape=(101, 1, ?) dtype=float32>,
 'prob_perplexity': <tf.Tensor 'wav2vec2/GumbelVectorQuantizer/Sum_3:0' shape=() dtype=float32>,
 'code_perplexity': <tf.Tensor 'wav2vec2/GumbelVectorQuantizer/Sum_1:0' shape=() dtype=float32>}

In [10]:
logits = r['x']
logits = tf.transpose(logits, [2, 1, 0])
logits = tf.reshape(logits, (-1, tf.shape(logits)[-1]))
target = tf.zeros(shape = (tf.shape(r['x'])[1] * tf.shape(r['x'])[2]), dtype = tf.int32)
logits, target

(<tf.Tensor 'Reshape:0' shape=(?, ?) dtype=float32>,
 <tf.Tensor 'zeros:0' shape=(?,) dtype=int32>)

In [11]:
loss = tf.nn.sparse_softmax_cross_entropy_with_logits(labels = target, logits = logits)
loss = tf.reduce_sum(loss)

In [12]:
sample_size = tf.cast(tf.shape(target)[0], tf.float32)

In [13]:
loss += (0.1 * (num_vars - r["prob_perplexity"]) / num_vars) * sample_size

In [14]:
optimizer = tf.train.AdamOptimizer().minimize(loss)

In [15]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

In [16]:
y, sr = malaya_speech.load('../speech/example-speaker/shafiqah-idayu.wav')
len(y) / sr

3.518625

In [17]:
import numpy as np

y1 = np.random.normal(size = (16000 * 10))

In [18]:
%%time

o = sess.run([r, logits, target, loss, optimizer], feed_dict = {i: y})

CPU times: user 18.8 s, sys: 1.01 s, total: 19.8 s
Wall time: 18.5 s


In [19]:
o[-2]

417.414

In [20]:
saver = tf.train.Saver()
saver.save(sess, 'test/model.ckpt')

'test/model.ckpt'

In [21]:
!ls -lh test
!rm -rf test

total 627280
-rw-r--r--  1 huseinzolkepli  staff    77B Apr 23 18:16 checkpoint
-rw-r--r--  1 huseinzolkepli  staff   280M Apr 23 18:16 model.ckpt.data-00000-of-00001
-rw-r--r--  1 huseinzolkepli  staff    69K Apr 23 18:16 model.ckpt.index
-rw-r--r--  1 huseinzolkepli  staff    18M Apr 23 18:16 model.ckpt.meta
